In [38]:
import os

In [39]:
%pwd

'/home/saitama/deepl_chicken_disease_classification/research'

In [40]:
os.chdir("../")

In [41]:
%pwd

'/home/saitama/deepl_chicken_disease_classification'

In [42]:
# configuration entity for data ingestion class

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir : Path
    tensorboard_root_log_dir : Path
    checkpoint_model_filepath : Path

In [43]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [44]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])




    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        
        model_checkpoints_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([
            Path(config.tensorboard_root_log_dir),
            Path(model_checkpoints_dir)
            ])

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )
        
        return prepare_callbacks_config




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")

        create_directories([training.root_dir])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )
   
        
        return training_config

In [45]:
import time

In [46]:
class PrepareCallbacks:
    def __init__(self, config= PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tensorboard_callbacks(self):
        timestamps = time.strftime("%Y-%m-%d-%H-%M-%S")
        tensorboard_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir,
                                          f"tb_log_at_{timestamps}",
                                          )
        return tf.keras.callbacks.TensorBoard(log_dir=tensorboard_running_log_dir)
    


    @property
    def _create_checkpoint_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(filepath= str(self.config.checkpoint_model_filepath),
                                                  save_best_only= True
                                                  )
    

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tensorboard_callbacks,
            self._create_checkpoint_callbacks
        ]


In [47]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [48]:
class Training:
    def __init__(self, config : TrainingConfig):
        self.config = config


    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.2
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        # generate validation data 
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        # generate data for training
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    
    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            callbacks=callback_list,
            validation_data = self.valid_generator,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [50]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    list_callbacks = prepare_callbacks.get_tb_ckpt_callbacks()


    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callback_list=list_callbacks)
except Exception as e:
    raise e

[2023-10-11 15:39:46,413: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-11 15:39:46,430: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-11 15:39:46,433: INFO: common: created directory at: artifacts]
[2023-10-11 15:39:46,439: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]
[2023-10-11 15:39:46,441: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2023-10-11 15:39:46,446: INFO: common: created directory at: artifacts/training]


Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Epoch 1/5
19/19 [==============================] - 223s 12s/step - loss: 12.7080 - accuracy: 0.5507 - val_loss: 7.3487 - val_accuracy: 0.3906
Epoch 2/5
19/19 [==============================] - 226s 12s/step - loss: 9.3400 - accuracy: 0.6047 - val_loss: 4.6947 - val_accuracy: 0.5469
Epoch 3/5
19/19 [==============================] - 221s 12s/step - loss: 7.0021 - accuracy: 0.6655 - val_loss: 14.1184 - val_accuracy: 0.6094
Epoch 4/5
19/19 [==============================] - 196s 10s/step - loss: 4.1763 - accuracy: 0.7669 - val_loss: 0.9615 - val_accuracy: 0.9062
Epoch 5/5
19/19 [==============================] - 217s 12s/step - loss: 0.8798 - accuracy: 0.8851 - val_loss: 3.4418 - val_accuracy: 0.7188
